In [3]:
import requests
import urllib
import pandas as pd

In [116]:
#先抓大商品標題&內容
keyword = '背心女'
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'x-api-source': 'pc',
    'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
}

s = requests.Session()
#url = 'https://cf.shopee.tw/file/a36d73057b95a451f6986689cca7cfc2'
#r = s.get(url, headers=headers)

base_url = 'https://shopee.tw/api/v4/search/search_items'
query = f"by=relevancy&keyword={keyword}&limit=50&newest=0&order=desc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2"
url = base_url + '?' + query
r = s.get(url, headers=headers)
if r.status_code == requests.codes.ok:
    data = r.json()
    # with open('shopee.json', 'w', encoding='utf-8') as f:
    #     f.write(r.text)

In [129]:
df = pd.json_normalize(data['items'])
df.columns
df.to_csv('shoppee_products.csv', encoding='utf_8_sig')
df_products = df[['itemid','shopid','item_basic.name']]
df_products.columns = ['itemid','shopid','name']
print(df_products.head())

       itemid     shopid                                               name
0  5383817226  338734079  【熱銷】【買一送一】【三套以上更優惠】專業無痕背心 冰絲背心 大尺碼背心 薄款打底 單穿百搭...
1  8057268321  409433252  【限時促銷】純棉吊帶 背心女 大碼韓版外穿寬鬆 夏季新款極簡文藝百搭內搭工字吊帶 打底無袖T...
2  2939150375  361800542  【低價爆款】在台現貨🌸 春夏新款純色內搭打底衫工字吊帶背心女性感外穿港味印花無袖T恤无袖韩版...
3  4891836032  455538747                      2021夏純色小吊帶背心打底衫內搭大碼圓領百搭性感小吊帶女
4  3586910583  421788737  女士無痕背心外穿吊帶衫內搭打底短款冰絲潮性感無袖針織性感夏加肥大尺碼200斤胖妹妹小可愛背心...


In [199]:
#再抓進入商品頁細部資訊&評論
#回傳資料裡面並無"商品網址"，要自己串#shopid：賣場ID。itemid：商品ID。
df1 = pd.DataFrame(columns=['itemid', 'rating', 'liked', 'videos', 'has_template_tag', 'shopid',
       'show_reply', 'rating_star', 'like_count', 'mtime', 'mentioned',
       'is_hidden', 'template_tags', 'author_portrait', 'detailed_rating',
       'orderid', 'cmtid', 'exclude_scoring_due_low_logistic', 'editable_date',
       'sync_to_social_toggle', 'opt', 'status', 'author_username', 'tags',
       'images', 'delete_operator', 'editable', 'anonymous', 'loyalty_info',
       'ctime', 'product_items', 'author_shopid', 'userid', 'comment',
       'filter', 'sync_to_social', 'delete_reason', 'ItemRatingReply.orderid',
       'ItemRatingReply.itemid', 'ItemRatingReply.cmtid',
       'ItemRatingReply.ctime', 'ItemRatingReply.mentioned',
       'ItemRatingReply.rating', 'ItemRatingReply.editable',
       'ItemRatingReply.userid', 'ItemRatingReply.shopid',
       'ItemRatingReply.comment', 'ItemRatingReply.filter',
       'ItemRatingReply.rating_star', 'ItemRatingReply.status',
       'ItemRatingReply.mtime', 'ItemRatingReply.opt',
       'ItemRatingReply.is_hidden'])

for itemid, shopid, name in products_list:
    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
        'x-api-source': 'pc',
        'referer': f'https://shopee.tw/{urllib.parse.quote(name)}-i.{urllib.parse.quote(str(shopid))}.{urllib.parse.quote(str(itemid))}'
    }

    s = requests.Session()
    #url = 'https://shopee.tw/api/v4/platform/get_ft_v2'
    #r = s.get(url, headers=headers)

    base_url = 'https://shopee.tw/api/v2/item/get_ratings'
    offset = 0
    for i in range(10): #假設最多10*6個評論
        if i > 0:
            offset = offset+6
        query = f"filter=0&flag=1&itemid={itemid}&limit=6&offset=0&shopid={shopid}&type=0"
        # https://shopee.tw/api/v2/item/get_ratings?filter=0&flag=1&itemid=5243593854&limit=6&offset=0&shopid=169621818&type=0
        url = base_url + '?' + query
        r = s.get(url, headers=headers)
        if r.status_code == requests.codes.ok:
            data2 = r.json()
            df2 = pd.json_normalize(data2['data']['ratings'])
            df1 = pd.concat([df1, df2], axis=0)
            # with open('shopee.json', 'w', encoding='utf-8') as f:
            #     f.write(r.text)

In [196]:
#單個商品測試(可略不看)
#再抓進入商品頁細部資訊&評論
#回傳資料裡面並無"商品網址"，要自己串#shopid：賣場ID。itemid：商品ID。
# df1 = pd.DataFrame(columns=['itemid', 'rating', 'liked', 'videos', 'has_template_tag', 'shopid',
#        'show_reply', 'rating_star', 'like_count', 'mtime', 'mentioned',
#        'is_hidden', 'template_tags', 'author_portrait', 'detailed_rating',
#        'orderid', 'cmtid', 'exclude_scoring_due_low_logistic', 'editable_date',
#        'sync_to_social_toggle', 'opt', 'status', 'author_username', 'tags',
#        'images', 'delete_operator', 'editable', 'anonymous', 'loyalty_info',
#        'ctime', 'product_items', 'author_shopid', 'userid', 'comment',
#        'filter', 'sync_to_social', 'delete_reason', 'ItemRatingReply.orderid',
#        'ItemRatingReply.itemid', 'ItemRatingReply.cmtid',
#        'ItemRatingReply.ctime', 'ItemRatingReply.mentioned',
#        'ItemRatingReply.rating', 'ItemRatingReply.editable',
#        'ItemRatingReply.userid', 'ItemRatingReply.shopid',
#        'ItemRatingReply.comment', 'ItemRatingReply.filter',
#        'ItemRatingReply.rating_star', 'ItemRatingReply.status',
#        'ItemRatingReply.mtime', 'ItemRatingReply.opt',
#        'ItemRatingReply.is_hidden'])

# itemid = '4551216182'
# shopid = '301395882'
# name = '【低價爆款】在台現貨🌸 春夏新款純色內搭打底衫工字吊帶背心女性感外穿港味印花無袖T恤无袖韩版女生小可愛 修身顯瘦 吊帶'
# headers = {
#     'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
#     'x-api-source': 'pc',
#     'referer': f'https://shopee.tw/{urllib.parse.quote(name)}-i.{urllib.parse.quote(shopid)}.{urllib.parse.quote(itemid)}'
# }

# s = requests.Session()
# #url = 'https://shopee.tw/api/v4/platform/get_ft_v2'
# #r = s.get(url, headers=headers)

# base_url = 'https://shopee.tw/api/v2/item/get_ratings'
# offset = 0
# for i in range(10): #假設最多60*6個評論
#     if i > 0:
#         offset = offset+6
#     query = f"filter=0&flag=1&itemid={itemid}&limit=6&offset={offset}&shopid={shopid}&type=0" #設定一次出現6筆評論
#     # https://shopee.tw/api/v2/item/get_ratings?filter=0&flag=1&itemid=5243593854&limit=6&offset=0&shopid=169621818&type=0
#     url = base_url + '?' + query
#     r = s.get(url, headers=headers)
#     if r.status_code == requests.codes.ok:
#         data2 = r.json()
#         if data2['data'] is not None:
#             df2 = pd.json_normalize(data2['data']['ratings'])
#             df1 = pd.concat([df1, df2], axis=0)
        # with open('shopee.json', 'w', encoding='utf-8') as f:
        #     f.write(r.text)

In [201]:
df1.to_csv('shoppee_comment_test.csv', encoding='utf_8_sig')